In [1]:
import tensorflow as tf
import PIL.Image as Image
import os
import numpy as np

2022-10-01 01:51:56.469679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-01 01:51:56.654852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 01:51:56.654887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-01 01:51:56.680030: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-01 01:51:57.488578: W tensorflow/stream_executor/platform/de

In [2]:
def join_images(data):
    mem = []
    for d in data:
        [passport, image, isSame] = d
        concatedImage = tf.concat([passport, image], 1)
        mem.append([concatedImage, isSame])
    return mem

In [3]:
def get_dataset(data):
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    x = np.array(x)
    y = np.array(y)
    return x,y    

In [9]:
def loadImages(path, size=128):
    peopleNames = os.listdir(path)
    images = []
    cmpr = True
    for name in peopleNames:
        if name == "vallari_x":
            cmpr = False
        else:
            cmpr = True
        currPath = os.path.join(path, name)
        currImagesDir = os.listdir(currPath)
        if(currImagesDir[0].split(".")[0] == "a"):
            image = currImagesDir[0]
            passport = currImagesDir[1]
        else:
            image = currImagesDir[1]
            passport = currImagesDir[0]
        p = Image.open(os.path.join(currPath, passport)).resize((size,size))
        i = Image.open(os.path.join(currPath, image)).resize((size,size))
        images.append([p, i, 1 if cmpr else 0])
    concatenated_image_data = join_images(images)
    x, y = get_dataset(concatenated_image_data)
    return x,y

In [10]:
x, y = loadImages("data/", 128)

In [11]:
y

array([0, 1, 1, 1])

In [12]:
def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 256, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [13]:
model = build_model()

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 254, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 127, 32)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 125, 64)       18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 60, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 107520)           

In [16]:
model.fit(x, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 620ms/step - loss: 57.6343 - accuracy: 0.2500
Epoch 2/10
1/1 [==============================] - 0s 145ms/step - loss: 987.2571 - accuracy: 0.7500
Epoch 3/10
1/1 [==============================] - 0s 146ms/step - loss: 565.1351 - accuracy: 0.7500
Epoch 4/10
1/1 [==============================] - 0s 143ms/step - loss: 115.4483 - accuracy: 0.7500
Epoch 5/10
1/1 [==============================] - 0s 140ms/step - loss: 237.8042 - accuracy: 0.2500
Epoch 6/10
1/1 [==============================] - 0s 142ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 139ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 137ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 148ms/step - loss: 1.6061e-04 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 164ms/step - loss: 1.2200e-24 - accurac

In [17]:
model.evaluate(x, y)

1/1 [==============================] - 0s 166ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

In [18]:
model.predict(x)

1/1 [==============================] - 0s 105ms/step


array([[0.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [19]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets
